In [3]:
# notebooks/analyze.ipynb
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns

# Đảm bảo hiển thị đồ thị trong Jupyter
%matplotlib inline

# Khởi tạo SparkSession
try:
    spark = SparkSession.builder \
        .appName("StockSentimentAnalysis") \
        .config("spark.mongodb.input.uri", "mongodb://mongodb:27017/stock_db.processed_news") \
        .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
        .master("spark://spark-master:7077") \
        .getOrCreate()
    print("SparkSession khởi tạo thành công!")
except Exception as e:
    print(f"Lỗi khi khởi tạo SparkSession: {e}")
    raise

# Tải dữ liệu đã xử lý từ MongoDB
try:
    df = spark.read.format("mongo") \
        .option("database", "stock_db") \
        .option("collection", "processed_news") \
        .load()
    print("Đọc dữ liệu từ MongoDB thành công!")
except Exception as e:
    print(f"Lỗi khi đọc dữ liệu từ MongoDB: {e}")
    raise

# In schema để kiểm tra các cột
print("Schema của DataFrame:")
df.printSchema()

# Hiển thị dữ liệu mẫu
print("Dữ liệu mẫu (5 dòng):")
df.show(5)

# Đếm số lượng tin tức
total_articles = df.count()
print(f"Tổng số bài viết: {total_articles}")

# Kiểm tra và phân loại sentiment
if total_articles > 0:
    if 'sentiment' in df.columns:
        # Phân loại sentiment
        sentiment_counts = df.groupBy("sentiment").count().toPandas()
        sentiment_counts['sentiment'] = sentiment_counts['sentiment'].map({1: 'Tích cực', 0: 'Tiêu cực'})
        print("Số lượng sentiment:")
        print(sentiment_counts)

        # Trực quan hóa
        plt.figure(figsize=(8, 6))
        sns.barplot(x='sentiment', y='count', data=sentiment_counts)
        plt.title("Phân bố Sentiment của tin tức chứng khoán")
        plt.xlabel("Sentiment")
        plt.ylabel("Số lượng bài viết")
        plt.show()
    else:
        print("Cột 'sentiment' không tồn tại trong dữ liệu. Kiểm tra lại collection 'processed_news' trong MongoDB.")
else:
    print("Không có dữ liệu để phân tích. Collection 'processed_news' có thể rỗng.")

# Dừng SparkSession
spark.stop()

SparkSession khởi tạo thành công!
Đọc dữ liệu từ MongoDB thành công!
Schema của DataFrame:
root

Dữ liệu mẫu (5 dòng):
++
||
++
++

Tổng số bài viết: 0
Không có dữ liệu để phân tích. Collection 'processed_news' có thể rỗng.
